In [ ]:
using Symbolics
using RuntimeGeneratedFunctions
RuntimeGeneratedFunctions.init(@__MODULE__)

In [ ]:
_prod(a, b, c, arrs...) = a .* _prod(b, c, arrs...)
_prod(a, b) = a .* b
_prod(a) = a

" makes polynomial combinations of basis "
function hamiltonian_poly(z, order, inds...)
    ham = []

    if order == 0
        Num(1)
    elseif order == length(inds)
        ham = vcat(ham, _prod([z[i] for i in inds]...))
    else
        start_ind = length(inds) == 0 ? 1 : inds[end]
        for j in start_ind:length(z)
            ham = vcat(ham, hamiltonian_poly(z, order, inds..., j))
        end
    end

    return ham
end

In [ ]:
" collects and sums only polynomial combinations of basis "
function hamiltonian(z, a, order)
    ham = []

    for i in 1:order
        ham = vcat(ham, hamiltonian_poly(z, i))
    end

    sum(collect(a .* ham))
end

In [ ]:
" collects and sums polynomial and trigonometric combinations of basis "
function hamil_trig(z, a, order, trig_wave_num)
    ham = []

    # Polynomial basis
    for i in 1:order
        ham = vcat(ham, hamiltonian_poly(z, i))
    end

    # Trigonometric basis
    for k = 1:trig_wave_num
        ham = vcat(ham, vcat(sin.(k*z)), vcat(cos.(k*z)))
    end

    ham = sum(collect(a .* ham))

    return ham

end

In [ ]:

" collects and sums polynomial, trigonometric, and states differences combinations of basis "
function hamiltonian_two(z, a, order, trig_wave_num, diffs_power, trig_state_diffs, exp_diff)
    ham = []

    # Polynomial basis
    for i in 1:order
        ham = vcat(ham, hamiltonian_poly(z, i))
    end

    # Trigonometric basis
    for k = 1:trig_wave_num
        ham = vcat(ham, vcat(sin.(k*z)), vcat(cos.(k*z)))
    end

    # For States difference power basis, trigonometric or exponential power states difference basis
    if diffs_power != 0 || trig_state_diffs != 0 || exp_diff != 0
        diffs = Vector{Symbolics.Num}()
        idx = 1
        for i in eachindex(z)
            for j in eachindex(z)
                if i == j
                    continue  # skip index where difference is between same state
                end
                push!(diffs, (z[i] - z[j]))
                idx += 1
            end
        end
    end
        
    if diffs_power > 0
        for k = 1:diffs_power
            ham = vcat(ham, vcat(diffs .^ k))
        end
    elseif diffs_power < 0
        for k = 1:abs(diffs_power)
            ham = vcat(ham, vcat(diffs .^ -k))
        end
    end

    # Trigonometric state differences basis
    if trig_state_diffs > 0
        for k = 1:trig_state_diffs
            ham = vcat(ham, vcat(sin.(diffs) .^ k), vcat(cos.(diffs) .^ k))
        end
    elseif trig_state_diffs < 0
        for k = 1:abs(trig_state_diffs)
            ham = vcat(ham, vcat(sin.(diffs) .^ -k), vcat(cos.(diffs) .^ -k))
        end
    end

    # exponential state differences basis
    if exp_diff > 0
        for k = 1:exp_diff
            ham = vcat(ham, vcat(exp.(diffs) .^ k))
        end
    elseif exp_diff < 0
        for k = 1:abs(exp_diff)
            ham = vcat(ham, vcat(exp.(diffs) .^ -k))
        end
    end

    ham = sum(collect(a .* ham))
    return ham

end

In [ ]:

"""
returns the number of required parameters taking into account many types of basis functions
"""
function calculate_nparams(nd, polyorder, trig_wave_num, diffs_power, trig_state_diffs, exp_diff)
    # binomial used to get the combination of polynomials till the highest order without repeat, e.g nparam = 34 for 3rd order, with z = q,p each of 2 dims
    # nd: total number of dims of all variable states
    nparam = binomial(nd + polyorder, polyorder) - 1

    if trig_wave_num > 0
        # first 2 in the product formula b/c the trig basis are sin and cos i.e. two basis functions
        nparam += 2 * trig_wave_num * nd
    end

    if abs(diffs_power) > 0
        # diffs power is the max power of the difference of states in the library of basis functions
        nparam += abs(diffs_power) * nd * (nd-1)
    end

    if abs(trig_state_diffs) > 0 
        # we add this b/c we also want to get the powers of sin and cos of the difference of states in the function library
        nparam += 2 * abs(trig_state_diffs) * nd * (nd-1)
    end

    if abs(exp_diff) > 0
        # diffs power is the max power of the difference of states in the library of basis functions
        nparam += abs(exp_diff) * nd * (nd-1)
    end

    return nparam
end

In [ ]:
" returns a function that can build the gradient of the hamiltonian "
function ΔH_func_builder_two(d, polyorder, trig_wave_num, diffs_power, trig_state_diffs, exp_diff)
    # nd is the total number of dimensions of all the states, e.g. if q,p each of 3 dims, that is 6 dims in total
    nd = 2d
    
    # binomial used to get the combination of variables till the highest order without repeat, nparam = 34 for 3rd order, with z = q,p each of 2 dims
    nparam = calculate_nparams(nd, polyorder, trig_wave_num, diffs_power, trig_state_diffs, exp_diff)

    # symbolic variables
    @variables a[1:nparam]
    @variables q[1:d]
    @variables p[1:d]
    z = vcat(q,p)
    Dz = Differential.(z)
    
    # make a basis library
    ham = hamiltonian_two(z, a, polyorder, trig_wave_num, diffs_power, trig_state_diffs, exp_diff)
    
    # gives derivative of the hamiltonian, but not the skew-symmetric true one
    f = [expand_derivatives(dz(ham)) for dz in Dz]

    # line below makes the vector into a hamiltonian vector field by multiplying with the skew-symmetric matrix
    ∇H = vcat(f[d+1:2d], -f[1:d])

    # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
    ∇H_eval = @RuntimeGeneratedFunction(Symbolics.inject_registered_module_functions(build_function(∇H, z, a)[2]))
    
    return ∇H_eval

end


In [ ]:

" returns a function that can build the gradient of the hamiltonian "
function hamilGrad_func_builder(d, polyorder, trig_wave_num, diffs_power, trig_state_diffs)
    # nd is the total number of dimensions of all the states, e.g. if q,p each of 3 dims, that is 6 dims in total
    nd = 2d
    # binomial used to get the combination of variables till the highest order without repeat, nparam = 34 for 3rd order, with z = q,p each of 2 dims
    nparam = calculate_nparams(nd, polyorder, trig_wave_num, diffs_power, trig_state_diffs)

    # symbolic variables
    @variables a[1:nparam]
    @variables q[1:d]
    @variables p[1:d]
    z = vcat(q,p)

    # usesine: whether to add trig basis or not
    if trig_wave_num > 0

        # gives derivative of the hamiltonian, but not the skew-symmetric true one
        Dz = Differential.(z)
        ∇H_add_trig = [expand_derivatives(dz(hamil_trig(z, a, polyorder, trig_wave_num))) for dz in Dz]

        # line below makes the vector into a hamiltonian vector field by multiplying with the skew-symmetric matrix
        ∇H_trig = vcat(∇H_add_trig[d+1:2d], -∇H_add_trig[1:d])

        # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
        ∇H_eval = @RuntimeGeneratedFunction(Symbolics.inject_registered_module_functions(build_function(∇H_trig, z, a)[2]))

        return ∇H_eval

    else

        # gives derivative of the hamiltonian, but not the skew-symmetric true one
        Dz = Differential.(z)
        f = [expand_derivatives(dz(hamiltonian(z, a, polyorder))) for dz in Dz]

        # line below makes the vector into a hamiltonian by multiplying with the skew-symmetric matrix
        ∇H = vcat(f[d+1:2d], -f[1:d])

        # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
        ∇H_eval = @RuntimeGeneratedFunction(Symbolics.inject_registered_module_functions(build_function(∇H, z, a)[2]))
        
        return ∇H_eval

    end

end

#### TESTING TIME ####

In [ ]:
# nd: total dims of all variables i.e p,q
nd = 4
# since we always have q and p, i.e 2 variables only, d will always be nd/2
d = div(nd, 2)

" trig_wave_num can be adjusted if higher frequency arguments expected "
trig_wave_num = 0

# highest order of polynomial library function
polyorder = 1

# states difference power
diffs_power = 0

# trig state difference power
trig_state_diffs = 0

# exponential state difference power
exp_diff = 1

"binomial used to get the combination of variables till the highest 
order without repeat, e.g with usesine= false, nparam = 34 for 3rd 
order, with z = q,p each of 2 dims"

nparam = calculate_nparams(nd, polyorder, trig_wave_num, diffs_power, trig_state_diffs, exp_diff)
println(nparam)

In [ ]:
# builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
∇H_eval = ΔH_func_builder_two(nd, polyorder, trig_wave_num, diffs_power, trig_state_diffs, exp_diff)

In [ ]:
# wrapper function for generalized SINDY hamiltonian gradient
function hamilGradient_general!(out, z, a::AbstractVector{T}, t) where T
    ∇H_eval(out, z, a)
    return out
end

In [ ]:
# 2D system with 4 variables [q₁, q₂, p₁, p₂] where q₂ = 0 and p₂ = 0
nd = 4

# 2 dims each of p and q gives 2*d = 4 variables
out = zeros(nd)

# let (a) be a vector of zeros initially of length 34 (b/c 34 is number of poly combinations for 2 variables, with 2 dims of highest order 3)

##################### NOTE: IN ACTUAL SCRIPT WE INITIALIZE THIS TO ZERO TO ALLOW BETTER OPTIMIZATION
a = ones(nparam)

x₀ = [2, 0, 0, 0]

t = 0
    

In [ ]:
hamilGradient_general!(out, x₀, a, 0)

#### New system for coefficients and basis ####

In [ ]:
function get_z_vector(dims)
    @variables q[1:dims]
    @variables p[1:dims]
    z = vcat(q,p)
    return z
end

In [ ]:
# make combinations of bases of just the order that is given 
# e.g order = 2 will give just the bases whose powers sum to 2
function poly_combos(z, order, inds...)
    if order == 0
        return Num[1]
    elseif order == length(inds)
        return [_prod([z[i] for i in inds]...)]
    else
        start_ind = length(inds) == 0 ? 1 : inds[end]
        return vcat([poly_combos(z, order, inds..., j) for j in start_ind:length(z)]...)
    end
end

# gives all bases monomials up to a certain order
function primal_monomial_basis(z, order::Int)
    return Vector{Symbolics.Num}(vcat([poly_combos(z, i) for i in 1:order]...))
end

# calculates coefficient bases up to a certain order
# mostly for use with trigonometric functions example sin(k*z),
# where k is the coefficient
function primal_coeff_basis(z, max_coeff::Int)
    return Vector{Symbolics.Num}(vcat([k .* z for k in 1:max_coeff]...))
end

# calculates +,-,*,/ between states as a new basis
# the return output is a set to avoid duplicates
function primal_operator_basis(z, operator)
    return Vector{Symbolics.Num}([operator(z[i], z[j]) for i in 1:length(z)-1 for j in i+1:length(z)] ∪ [operator(z[j], z[i]) for i in 1:length(z)-1 for j in i+1:length(z)])
end

# calculates power of states as a new basis
function primal_power_basis(z, max_power::Int)
    if max_power > 0
        return Vector{Symbolics.Num}(vcat([z.^i for i in 1:max_power]...))
    elseif max_power < 0
        return Vector{Symbolics.Num}(vcat([z.^-i for i in 1:abs(max_power)]...))
    end
end

In [ ]:
function polynomial_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = primal_monomial_basis(z, polyorder)
    primes = vcat(primes, primal_coeff_basis(z, max_coeff))
    if operator !== nothing
        primes = vcat(primes, primal_operator_basis(z, operator))
    end
    return primes
end

function trigonometric_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    return vcat(sin.(primes), cos.(primes))
end

function exponential_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    return exp.(primes)
end

function logarithmic_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    return log.(primes)
end

function mixed_states_basis(basis::Vector{Symbolics.Num}...)
    mixed_states = Tuple(basis)
    
    ham = Vector{Symbolics.Num}()
    for i in eachindex(mixed_states)
        for j in i+1:lastindex(mixed_states)
            ham = vcat(ham, [mixed_states[i][k] * mixed_states[j][l] for k in 1:length(mixed_states[i]) for l in 1:length(mixed_states[j])])
        end
    end
    
    return Vector{Symbolics.Num}(ham)
end

In [ ]:
# Returns the number of required coefficients for the bases
function get_numCoeffs(basis::Vector{Symbolics.Num})
    return length(basis)
end

In [ ]:
# gets a vector of combinations of basis
function get_basis_set(bases::Vector{Symbolics.Num}...)
    # gets a vector of combinations of basis
    basis = vcat(bases...)
    
    # removes duplicates
    basis = Vector{Symbolics.Num}(collect(Set(basis)))

    return basis
end

In [ ]:
function ΔH_func_builder(d::Int, z::Vector{Symbolics.Num} = get_z_vector(d), bases::Vector{Symbolics.Num}...) 
    # nd is the total number of dimensions of all the states, e.g. if q,p each of 3 dims, that is 6 dims in total
    nd = 2d
    Dz = Differential.(z)
    
    # collects and sums combinations of basis and coefficients"
    basis = get_basis_set(bases...)
   
    # gets number of terms in the basis
    @variables a[1:get_numCoeffs(basis)]
    
    # collect and sum combinations of basis and coefficients
    ham = sum(collect(a .* basis))
    
    # gives derivative of the hamiltonian, but not the skew-symmetric true one
    f = [expand_derivatives(dz(ham)) for dz in Dz]
    
    # line below makes the vector into a hamiltonian vector field by multiplying with the skew-symmetric matrix
    ∇H = vcat(f[d+1:2d], -f[1:d])
    
    # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
    ∇H_eval = @RuntimeGeneratedFunction(Symbolics.inject_registered_module_functions(build_function(∇H, z, a)[2]))
    
    return ∇H_eval
end

In [ ]:
d = 1
z = get_z_vector(d)
z = primal_operator_basis(z, -)
polynomial = polynomial_basis(z, polyorder=2)
trigonometric  = trigonometric_basis(z, polyorder = 1, operator = /)
logarithmic = logarithmic_basis(z, polyorder = 0, operator = -)
exponential = exponential_basis(z, polyorder=1, operator = +, max_coeff=1)
mixed_basis = mixed_states_basis(trigonometric, polynomial)
prime_mono = primal_monomial_basis(z, d)

nd = 2d
Dz = Differential.(z)

# collects and sums combinations of basis and coefficients"
basis = get_basis_set(polynomial)

# gets number of terms in the basis
@variables a[1:get_numCoeffs(basis)]

# collect and sum combinations of basis and coefficients
ham = sum(collect(a .* basis))

# gives derivative of the hamiltonian, but not the skew-symmetric true one
f = [expand_derivatives(dz(ham)) for dz in Dz]

# # line below makes the vector into a hamiltonian vector field by multiplying with the skew-symmetric matrix
# ∇H = vcat(f[d+1:2d], -f[1:d])

In [ ]:
d = 2
z = get_z_vector(d)
z = primal_operator_basis(z, -)
polynomial = polynomial_basis(z, polyorder=3)
trigonometric  = trigonometric_basis(z, polyorder = 1, operator = /)
logarithmic = logarithmic_basis(z, polyorder = 0, operator = -)
exponential = exponential_basis(z, polyorder=1, operator = +, max_coeff=1)
mixed_basis = mixed_states_basis(trigonometric, polynomial)

out = ΔH_func_builder(d, z,
    polynomial, trigonometric
    ) 

In [ ]:
z = get_z_vector(4)
polynomial = polynomial_basis(z, polyorder=2)

z_diff = primal_operator_basis(z, -)
exponential_diff  = exponential_basis(z_diff, polyorder=1)
z_power = primal_power_basis(exponential_diff, 2)
basis = get_basis_set(polynomial, z_power)
# basis = get_basis_set(polynomial)
# basis = get_basis_set(exponential_diff)
@variables a[1:get_numCoeffs(basis)]
nd = 8
Dz = Differential.(z)
# collect and sum combinations of basis and coefficients
ham = sum(collect(a .* basis))

# # gives derivative of the hamiltonian, but not the skew-symmetric true one
# f = [expand_derivatives(dz(ham)) for dz in Dz]

# # # line below makes the vector into a hamiltonian vector field by multiplying with the skew-symmetric matrix
# ∇H = vcat(f[d+1:2d], -f[1:d])

In [ ]:
ham = (collect(a .* basis))